In [19]:
import numpy as np
import torch
from multiprocessing import Pool

In [20]:
def compute_reg_nll(dets, gt, covar_e = None, covar_a = None, w=0.0):
    assert len(dets) == len(gt)
    dets_loc = torch.from_numpy(dets[:,:8])
    dets_loc = torch.reshape(dets_loc, (-1, 2))
    gt_loc = torch.from_numpy(gt)
    gt_loc = torch.reshape(gt_loc, (-1, 2))
    if len(dets[0]) <= 9:
        covar_matrix = covar_e
    else:
        dets_covar = torch.from_numpy(dets[:,9:])
        dets_covar = torch.reshape(dets_covar, (-1, 3))
        u_matrix = torch.zeros((dets_covar.shape[0], 2, 2))
        u_matrix[:, 0, 0] = torch.exp(dets_covar[:, 0])
        u_matrix[:, 0, 1] = dets_covar[:, 1]
        u_matrix[:, 1, 1] = torch.exp(dets_covar[:, 2])
        sigma_inverse = torch.matmul(torch.transpose(u_matrix, 1, 2), u_matrix)
        covar_matrix = torch.linalg.inv(sigma_inverse)
        if covar_e != None and covar_a != None:
            #covar_matrix = covar_e + w * ( 0.5*covar_a + 0.5* covar_matrix + 1e-2*torch.eye(2))
            #covar_matrix = covar_e + w * (0.5 * covar_a + 0.5 * covar_matrix) * 100.0
            #covar_matrix = w * covar_e + (1-w)*(0.5 * covar_a + 0.5 * covar_matrix)*1000.0
            covar_matrix = w * covar_e + (1-w)*(0.5 * covar_a + 0.5 * covar_matrix) * 100.0
        else:
            #covar_matrix = covar_matrix + 1e-2*torch.eye(2)
            covar_matrix = covar_matrix * 100.0
    predicted_multivariate_normal_dists = torch.distributions.multivariate_normal.MultivariateNormal(dets_loc, covariance_matrix = covar_matrix)
    negative_log_prob = - \
        predicted_multivariate_normal_dists.log_prob(gt_loc)
    #negative_log_prob_mean = negative_log_prob.mean()
    return negative_log_prob.tolist()

In [27]:
def get_cls_results(det_results, annotations, class_id):
    """Get det results and gt information of a certain class.
    Args:
        det_results (list[list]): Same as `eval_map()`.
        annotations (list[dict]): Same as `eval_map()`.
        class_id (int): ID of a specific class.
    Returns:
        tuple[list[np.ndarray]]: detected bboxes, gt bboxes, ignored gt bboxes
    """
    cls_dets = [img_res[class_id] for img_res in det_results]
    cls_gts = []
    cls_gts_ignore = []
    for ann in annotations:
        gt_inds = ann["labels"] == class_id

        if ann["bboxes"].ndim == 1:
            cls_gts.append(ann["bboxes"][0])
        else:
            cls_gts.append(ann["bboxes"][gt_inds, :])

        if ann.get("labels_ignore", None) is not None:
            ignore_inds = ann["labels_ignore"] == class_id

            if ann["bboxes_ignore"].ndim == 1:
                cls_gts_ignore.append(ann["bboxes_ignore"][0])
            else:
                cls_gts_ignore.append(ann["bboxes_ignore"][ignore_inds, :])
        else:
            cls_gts_ignore.append(np.empty((0, 8), dtype=np.float32))

    return cls_dets, cls_gts, cls_gts_ignore
def match_tp_fp(
    det_bboxes, gt_bboxes, gt_bboxes_ignore=None, iou_thr=0.5, area_ranges=None
):
    """Check if detected bboxes are true positive or false positive.
    Args:
        det_bbox (ndarray): Detected bboxes of this image, of shape (m, 5).
        gt_bboxes (ndarray): GT bboxes of this image, of shape (n, 4).
        gt_bboxes_ignore (ndarray): Ignored gt bboxes of this image,
            of shape (k, 4). Default: None
        iou_thr (float): IoU threshold to be considered as matched.
            Default: 0.5.
        area_ranges (list[tuple] | None): Range of bbox areas to be evaluated,
            in the format [(min1, max1), (min2, max2), ...]. Default: None.
    Returns:
        tuple[np.ndarray]: (tp, fp) whose elements are 0 and 1. The shape of
            each array is (num_scales, m).
    """
    # an indicator of ignored gts
    gt_ignore_inds = np.concatenate(
        (
            np.zeros(gt_bboxes.shape[0], dtype=np.bool),
            np.ones(gt_bboxes_ignore.shape[0], dtype=np.bool),
        )
    )
    # stack gt_bboxes and gt_bboxes_ignore for convenience
    gt_bboxes = np.vstack((gt_bboxes, gt_bboxes_ignore))

    num_dets = det_bboxes.shape[0]
    num_gts = gt_bboxes.shape[0]
    if area_ranges is None:
        area_ranges = [(None, None)]
        
    num_scales = len(area_ranges)
    # tp and fp are of shape (num_scales, num_gts), each row is tp or fp of
    # a certain scale
    tp = np.zeros((num_scales, num_dets), dtype=np.int)
    fp = np.zeros((num_scales, num_dets), dtype=np.int)

    # if there is no gt bboxes in this image, then all det bboxes
    # within area range are false positives
    if gt_bboxes.shape[0] == 0:
        if area_ranges == [(None, None)]:
            fp[...] = 1
        else:
            det_areas = (det_bboxes[:, 2] - det_bboxes[:, 0]) * (
                det_bboxes[:, 3] - det_bboxes[:, 1]
            )
            for i, (min_area, max_area) in enumerate(area_ranges):
                fp[i, (det_areas >= min_area) & (det_areas < max_area)] = 1
        return tp, fp

    gt_corners = np.zeros((gt_bboxes.shape[0], 4, 2), dtype=np.float32)
    pred_corners = np.zeros((det_bboxes.shape[0], 4, 2), dtype=np.float32)

    for k in range(gt_bboxes.shape[0]):
        gt_corners[k, 0, 0] = gt_bboxes[k][0]
        gt_corners[k, 0, 1] = gt_bboxes[k][1]
        gt_corners[k, 1, 0] = gt_bboxes[k][2]
        gt_corners[k, 1, 1] = gt_bboxes[k][3]
        gt_corners[k, 2, 0] = gt_bboxes[k][4]
        gt_corners[k, 2, 1] = gt_bboxes[k][5]
        gt_corners[k, 3, 0] = gt_bboxes[k][6]
        gt_corners[k, 3, 1] = gt_bboxes[k][7]

    if det_bboxes.ndim == 1:
        det_bboxes = np.array([det_bboxes])

    for k in range(det_bboxes.shape[0]):
        pred_corners[k, 0, 0] = det_bboxes[k][0]
        pred_corners[k, 0, 1] = det_bboxes[k][1]
        pred_corners[k, 1, 0] = det_bboxes[k][2]
        pred_corners[k, 1, 1] = det_bboxes[k][3]
        pred_corners[k, 2, 0] = det_bboxes[k][4]
        pred_corners[k, 2, 1] = det_bboxes[k][5]
        pred_corners[k, 3, 0] = det_bboxes[k][6]
        pred_corners[k, 3, 1] = det_bboxes[k][7]

    gt_box = convert_format(gt_corners)
    pred_box = convert_format(pred_corners)
    save_flag = False
    for gt in gt_box:
        iou = np.array(compute_iou(gt, pred_box))
        if not save_flag:
            box_iou = iou
            save_flag = True
        else:
            box_iou = np.vstack((box_iou, iou))

    # make dimension the same
    if len(gt_box) == 1:
        box_iou = np.array([box_iou])

    ious = box_iou.T
    #    ious = bbox_overlaps(det_bboxes, gt_bboxes)
    # for each det, the max iou with all gts
    ious_max = ious.max(axis=1)

    # for each det, which gt overlaps most with it
    ious_argmax = ious.argmax(axis=1)
    # sort all dets in descending order by scores
    sort_inds = np.argsort(-det_bboxes[:, 8])
    for k, (min_area, max_area) in enumerate(area_ranges):
        gt_covered = np.zeros(num_gts, dtype=bool)
        # if no area range is specified, gt_area_ignore is all False
        if min_area is None:
            gt_area_ignore = np.zeros_like(gt_ignore_inds, dtype=bool)
        else:
            gt_areas = (gt_bboxes[:, 2] - gt_bboxes[:, 0]) * (
                gt_bboxes[:, 3] - gt_bboxes[:, 1]
            )
            gt_area_ignore = (gt_areas < min_area) | (gt_areas >= max_area)
        for i in sort_inds:
            if ious_max[i] >= iou_thr:
                matched_gt = ious_argmax[i]
                if not (gt_ignore_inds[matched_gt] or gt_area_ignore[matched_gt]):
                    if not gt_covered[matched_gt]:
                        gt_covered[matched_gt] = True
                        tp[k, i] = matched_gt
                    else:
                        fp[k, i] = 1
                # otherwise ignore this detected bbox, tp = 0, fp = 0
            elif min_area is None:
                fp[k, i] = 1
            else:
                bbox = det_bboxes[i, :4]
                area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                if area >= min_area and area < max_area:
                    fp[k, i] = 1
    return tp, fp
def eval_nll(
    det_results,
    annotations,
    scale_ranges=None,
    iou_thr=0.5,
    logger=None,
    nproc=4,
    covar_e = None,
    covar_a = None,
    w=0.0
):
    """Evaluate mAP of a dataset.
    Args:
        det_results (list[list]): [[cls1_det, cls2_det, ...], ...].
            The outer list indicates images, and the inner list indicates
            per-class detected bboxes.
        annotations (list[dict]): Ground truth annotations where each item of
            the list indicates an image. Keys of annotations are:
            - `bboxes`: numpy array of shape (n, 4)
            - `labels`: numpy array of shape (n, )
            - `bboxes_ignore` (optional): numpy array of shape (k, 4)
            - `labels_ignore` (optional): numpy array of shape (k, )
        scale_ranges (list[tuple] | None): Range of scales to be evaluated,
            in the format [(min1, max1), (min2, max2), ...]. A range of
            (32, 64) means the area range between (32**2, 64**2).
            Default: None.
        iou_thr (float): IoU threshold to be considered as matched.
            Default: 0.5.
        dataset (list[str] | str | None): Dataset name or dataset classes,
            there are minor differences in metrics for different datsets, e.g.
            "voc07", "imagenet_det", etc. Default: None.
        logger (logging.Logger | str | None): The way to print the mAP
            summary. See `mmdet.utils.print_log()` for details. Default: None.
        nproc (int): Processes used for computing TP and FP.
            Default: 4.
    Returns:
        tuple: [[tp_nll, fp_entropy]]
    """
    assert len(det_results) == len(annotations)
    num_imgs = len(det_results)
    num_scales = len(scale_ranges) if scale_ranges is not None else 1
    num_classes = len(det_results[0])  # positive class num
    area_ranges = (
        [(rg[0] ** 2, rg[1] ** 2) for rg in scale_ranges]
        if scale_ranges is not None
        else None
    )

    pool = Pool(nproc)
    eval_results = []
    for i in range(num_classes):
        # get gt and det bboxes of this class
        cls_dets, cls_gts, cls_gts_ignore = get_cls_results(det_results, annotations, i)
        tp_nll = []
        fp_entropy = []
        tpfp_func = match_tp_fp
        # compute tp and fp for each image with multiple processes
        tpfp = pool.starmap(
            tpfp_func,
            zip(
                cls_dets,
                cls_gts,
                cls_gts_ignore,
                [iou_thr for _ in range(num_imgs)],
                [area_ranges for _ in range(num_imgs)],
            ),
        )
        tp_all, fp_all = list(zip(*tpfp))
        # calculate gt number of each scale
        # ignored gts or gts beyond the specific scale are not counted
        num_gts = np.zeros(num_scales, dtype=int)
        for j, bbox in enumerate(cls_gts):
            if area_ranges is None:
                num_gts[0] += bbox.shape[0]
            else:
                gt_areas = (bbox[:, 2] - bbox[:, 0]) * (bbox[:, 3] - bbox[:, 1])
                for k, (min_area, max_area) in enumerate(area_ranges):
                    num_gts[k] += np.sum((gt_areas >= min_area) & (gt_areas < max_area))
        # sort all det bboxes by score, also sort tp and fp
        for dets, gt, match, fp in zip(cls_dets, cls_gts, tp_all, fp_all):
            tp = np.squeeze((1 - fp).astype(bool))
            fp = np.squeeze(fp.astype(bool))
            match = np.squeeze(match)
            if tp.shape is ():
                tp = np.array([tp])
                match = np.array([match])
            if fp.shape is ():
                fp = np.array([fp])
            if len(tp) != 0:
                tp_dets = dets[tp]
                tp_match = match[tp]
                tp_gt = gt[tp_match]
                nll = compute_reg_nll(tp_dets, tp_gt, covar_e, covar_a, w)
                tp_nll.extend(nll)
            #if len(dets[fp]) != 0:
            #    fp_dets = dets[fp]
            #    entropy = compute_reg_entropy(fp_dets)
            #    fp_entropy.extend(entropy)
        eval_results.append({
            "num_gts": num_gts,
            "NLL": np.mean(tp_nll),
            #"FP_entropy": np.mean(fp_entropy)
        })
    return eval_results

In [28]:
def compute_one_nll(data_path, covar_path=None, iou_thr=0.7):
    data = np.load(data_path, allow_pickle=True)
    det_results_all_local = data.item()['det_results_frame']
    annotations_all_local = data.item()['annotations_frame']
    if covar_path != None:
        covar_data = np.load(covar_path, allow_pickle=True)
        covar_e = covar_data.item()['covar_e']
        covar_a = covar_data.item()['covar_a']
        covar_e = torch.from_numpy(covar_e)
        covar_a = torch.from_numpy(covar_a)
        print("covar_e:")
        print(covar_e)
        print("covar_a")
        print(covar_a)
        w = 0.5
    else:
        covar_a = None
        covar_e = None
        w = 0.0
    print("NLL:")
    covar_nll = eval_nll(det_results_all_local, annotations_all_local, scale_ranges=None, iou_thr=iou_thr, covar_e = covar_e, covar_a=covar_a, w=w)
    print(covar_nll)

def compute_null_with_different_weight(data_path, covar_path, iou_thr=0.7):
    data = np.load(data_path, allow_pickle=True)
    det_results_all_local = data.item()['det_results_frame']
    annotations_all_local = data.item()['annotations_frame']
    covar_data = np.load(covar_path, allow_pickle=True)
    covar_e = covar_data.item()['covar_e']
    covar_a = covar_data.item()['covar_a']
    covar_e = torch.from_numpy(covar_e)
    covar_a = torch.from_numpy(covar_a)
    print("covar_e:")
    print(covar_e)
    print("covar_a")
    print(covar_a)
    
    #w_list = np.arange(0.0, 30.0, 0.5)
    w_list = np.arange(0.0, 1.05, 0.05)
    #w_list = np.arange(1.0, 200.0, 1.0)
    #w_list = np.arange(0.2, 3.0, 0.2)
    nll_list = []
    for index, w in enumerate(w_list):
        covar_nll = eval_nll(det_results_all_local, annotations_all_local, scale_ranges=None, iou_thr=iou_thr, covar_e = covar_e, covar_a=covar_a, w=w)
        nll_list.append(covar_nll[0]['NLL'])
        print("w: {}, NLL: {}".format(w, covar_nll))
    return nll_list, w_list

In [30]:
#covar_matrix = w * covar_e + (1-w)*(0.5 * covar_a + 0.5 * covar_matrix) * 100.0
#IOU 0.5
disco_data_path = "./check/test_loss_mbb_two_step_corner_pair_ind/disco/no_rsu/23/all_data.npy"
disco_covar_path = "./check/check_loss_mbb_two_step_corner_pair_ind/disco/no_rsu/mbb_covar.npy"
torch.multiprocessing.set_sharing_strategy("file_system")
compute_one_nll(disco_data_path, disco_covar_path)

covar_e:
tensor([[ 3.7420, -0.0387],
        [-0.0387,  2.4557]], dtype=torch.float64)
covar_a
tensor([[1.0104e-03, 1.0640e-05],
        [1.0640e-05, 9.1226e-04]], dtype=torch.float64)
NLL:


/home/sas20048/anaconda/envs/coperception/lib/python3.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/sas20048/anaconda/envs/coperception/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/sas20048/anaconda/envs/coperception/lib/python3.7/site-packages/

NameError: name 'convert_format' is not defined

In [7]:
#covar_matrix = w * covar_e + (1-w)*(0.5 * covar_a + 0.5 * covar_matrix) * 100.0
#IOU 0.7
compute_one_nll(disco_data_path, disco_covar_path)

covar_e:
tensor([[ 3.7420, -0.0387],
        [-0.0387,  2.4557]], dtype=torch.float64)
covar_a
tensor([[1.0104e-03, 1.0640e-05],
        [1.0640e-05, 9.1226e-04]], dtype=torch.float64)
NLL:
[{'num_gts': array([50047]), 'NLL': 2.68561090312438}]


In [8]:
#covar_matrix = covar_matrix * 100.0
#IOU 0.7
compute_one_nll(disco_data_path)

NLL:
[{'num_gts': array([50047]), 'NLL': 7.896231654163999}]


In [11]:
#covar_matrix = covar_matrix * 10.0
#IOU 0.7
compute_one_nll(disco_data_path)

NLL:
[{'num_gts': array([50047]), 'NLL': 7.896231654163999}]


In [14]:
#covar_matrix = covar_matrix
#IOU 0.7
compute_one_nll(disco_data_path)

NLL:
[{'num_gts': array([50047]), 'NLL': 7.896231654163999}]
